In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import random
from tqdm import tqdm
import urllib2
import requests
from geopy.geocoders import Nominatim
import json 

darksky_key = 'd7ac916fc413d1657f252522d9563059'
matteus = 'c08fb197b592238160e4dc828ada0522'
base_url = 'https://api.darksky.net/forecast/c08fb197b592238160e4dc828ada0522/'
address = 'Muntstraat 10 Leuven'

In [ ]:
def url_builder(start_date):
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    url_list = []
    today = dt.date.today()
    delta = today - start_date
    for counter in range(delta.days):
        timestamp = str(time.mktime((start_date + dt.timedelta(days=counter)).timetuple()))[:-2]
        full_url = base_url + str(latitude) + ',' + str(longitude) + ',' + timestamp
        url_list.append(full_url)
    return url_list

In [ ]:
def f_t_c(fahrenheit):
    return (((fahrenheit - 32) * 5.0) / 9.0)

In [ ]:
list_url = url_builder(dt.date(2017,6,10))
list_url

In [ ]:
def build_json_from_start_date(start_date, filename):
    list_url = url_builder(start_date)
    json_file = open(filename, "w")
    for url in tqdm(list_url):     
        success = False
        while(not success):
            try:
                request = requests.get(url=url, timeout=1)
                if request is None:
                    while(request is None):
                        request = requests.get(url=url, timeout=1)
                content = json.loads(request.content)
                json_temp = content.get('hourly').get('data')
                for temp in json_temp:
                    #changing hourly keys
                    temp["HourlyTemperature"] = f_t_c(temp["temperature"])
                    del temp["temperature"]
                    temp["HourlyDewPoint"] = temp["dewPoint"]
                    del temp["dewPoint"]
                    temp["HourlySummary"] = temp["summary"]
                    del temp["summary"]
                    temp["HourlyApparentTemperature"] = f_t_c(temp["apparentTemperature"])
                    del temp["apparentTemperature"]
                    temp["HourlyPressure"] = temp["pressure"]
                    del temp["pressure"]
                    temp["HourlyWindSpeed"] = temp["windSpeed"]
                    del temp["windSpeed"]
                    temp["HourlyTime"] = temp["time"]
                    del temp["time"]
                    temp["HourlyWindBearing"] = temp["windBearing"]
                    del temp["windBearing"]
                    temp["HourlyHumidity"] = temp["humidity"]
                    del temp["humidity"]
                    temp["HourlyIcon"] = temp["icon"]
                    del temp["icon"]

                    #appending daily data
                    temp["DailyTemperatureMax"] = f_t_c(content.get("daily").get('data')[0]["temperatureMax"])
                    temp["DailyApparentTemperatureMinTime"] = content.get("daily").get('data')[0]["apparentTemperatureMinTime"]
                    temp["DailyTemperatureMin"] = f_t_c(content.get("daily").get('data')[0]["temperatureMin"])
                    temp["DailySummary"] = content.get("daily").get('data')[0]["summary"]
                    temp["DailyDewPoint"] = content.get("daily").get('data')[0]["dewPoint"]
                    temp["DailyApparentTemperatureMax"] = f_t_c(content.get("daily").get('data')[0]["apparentTemperatureMax"])
                    temp["DailyTemperatureMaxTime"] = content.get("daily").get('data')[0]["temperatureMaxTime"]
                    temp["DailyWindBearing"] = content.get("daily").get('data')[0]["windBearing"]
                    temp["DailyMoonPhase"] = content.get("daily").get('data')[0]["moonPhase"]
                    temp["DailySunsetTime"] = content.get("daily").get('data')[0]["sunsetTime"]
                    temp["DailyPressure"] = content.get("daily").get('data')[0]["pressure"]
                    temp["DailyApparentTemperatureMin"] = f_t_c(content.get("daily").get('data')[0]["apparentTemperatureMin"])
                    temp["DailyIcon"] = content.get("daily").get('data')[0]["icon"]
                    temp["DailyapparentTemperatureMaxTime"] = content.get("daily").get('data')[0]["apparentTemperatureMaxTime"]
                    temp["DailyHumidity"] = content.get("daily").get('data')[0]["humidity"]
                    temp["DailyWindSpeed"] = content.get("daily").get('data')[0]["windSpeed"]
                    temp["DailyTime"] = content.get("daily").get('data')[0]["time"]
                    temp["DailySunriseTime"] = content.get("daily").get('data')[0]["sunriseTime"]
                    temp["DailyTemperatureMinTime"] = content.get("daily").get('data')[0]["temperatureMinTime"]

                    json_file.write(json.dumps(temp) + '\n')
                    success = True
            except KeyError as e:
                success = False
                print "KeyError at url " + url + "\nWith error:\n\t" + str(e)
    json_file.close()
        

In [ ]:
build_json_from_start_date(start_date=dt.date(2014,10,1), filename="test_data.json")

In [ ]:
lis = url_builder(dt.date(2014,10,1))
print len(lis)

In [ ]:

request = requests.get(url='https://api.darksky.net/forecast/c08fb197b592238160e4dc828ada0522/50.8782678,4.7015134,1430604000', timeout=1)
if request is None:
    while(request is None):
        request = requests.get(url=url, timeout=1)
content = json.loads(request.content)
json_temp = content.get('hourly').get('data')
for temp in json_temp:
    print temp